We project our 2-year predictions for different stations onto all possible locations in Finland, with WindAtlas data

Justification: microclimates from geographical features can significantly affect wind speed in a location, we're capturing this with predictions done by WindAtlas that takes this into account

Method:
- load predictions from stations
- for all stations, sample windatlas and get coefficient between our prediction and windatlas one
- for all possible locations, get windatlas prediction and multiply by coefficient

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import math
import rioxarray
import rasterio
import joblib
from tqdm import tqdm
tqdm.pandas()

In [2]:
# Load weather predictions and stations
weather_stations = gpd.read_file('data/fmi/weather_stations.geojson')
weather_stations_proj = weather_stations.to_crs("EPSG:3067")
wind_pred = pd.read_csv('data/predictions/wind_preds.csv').rename(
    columns={"Station": "name", "Prediction": "wind_speed_pred"})
weather = weather_stations.merge(wind_pred, on="name", how='outer').dropna()
weather = weather[weather['wind_speed_pred'] > 0]
weather

,name,geometry,wind_speed_pred
1,Asikkala Pulkkilanharju,POINT (25.52021 61.26521),3.123616
3,Enontekiö Kilpisjärvi Saana,POINT (20.85091 69.04277),6.567162
4,Enontekiö Kilpisjärvi kyläkeskus,POINT (20.81379 69.03905),3.627882
5,Enontekiö Näkkälä,POINT (23.57595 68.60301),3.217094
6,Enontekiö lentoasema,POINT (23.42755 68.36140),2.996602
...,...,...,...
227,Virolahti Koivuniemi,POINT (27.67274 60.52720),2.454944
228,Virrat Äijänneva,POINT (23.54228 62.32783),2.449347
229,Ylitornio Meltosjärvi,POINT (24.64960 66.52952),3.015766
230,Ylivieska lentokenttä,POINT (24.72468 64.05029),2.662563


In [3]:
# Load wind observations
wind_obs = pd.read_csv('data/fmi/wind_speed_data.csv', index_col=0).iloc[-1].T
wind_obs.index.name = 'name'
wind_obs.name = 'wind_speed_obs'
weather = weather.merge(wind_obs, on="name", how='left').dropna()
weather

,name,geometry,wind_speed_pred,wind_speed_obs
0,Asikkala Pulkkilanharju,POINT (25.52021 61.26521),3.123616,3.3
1,Enontekiö Kilpisjärvi Saana,POINT (20.85091 69.04277),6.567162,7.3
2,Enontekiö Kilpisjärvi kyläkeskus,POINT (20.81379 69.03905),3.627882,1.0
3,Enontekiö Näkkälä,POINT (23.57595 68.60301),3.217094,3.5
4,Enontekiö lentoasema,POINT (23.42755 68.36140),2.996602,0.8
...,...,...,...,...
174,Viitasaari Haapaniemi,POINT (25.85862 63.08225),1.716407,1.1
175,Virolahti Koivuniemi,POINT (27.67274 60.52720),2.454944,4.4
176,Virrat Äijänneva,POINT (23.54228 62.32783),2.449347,1.1
177,Ylitornio Meltosjärvi,POINT (24.64960 66.52952),3.015766,1.6


In [4]:
# Load windatlas raster
wind_raster = rasterio.open("data/wind/FIN_wind-speed_100m.tif")
wind_raster.bounds


BoundingBox(left=19.084072682194545, bottom=58.845768747736074, right=31.581572682195166, top=70.08826874773663)

In [5]:
station_coords = [(p.x, p.y) for p in weather['geometry']]
weather['wind_speed_atlas'] = list(i[0] for i in wind_raster.sample(station_coords))
weather

,name,geometry,wind_speed_pred,wind_speed_obs,wind_speed_atlas
0,Asikkala Pulkkilanharju,POINT (25.52021 61.26521),3.123616,3.3,7.412938
1,Enontekiö Kilpisjärvi Saana,POINT (20.85091 69.04277),6.567162,7.3,9.285563
2,Enontekiö Kilpisjärvi kyläkeskus,POINT (20.81379 69.03905),3.627882,1.0,5.937158
3,Enontekiö Näkkälä,POINT (23.57595 68.60301),3.217094,3.5,6.891766
4,Enontekiö lentoasema,POINT (23.42755 68.36140),2.996602,0.8,6.528882
...,...,...,...,...,...
174,Viitasaari Haapaniemi,POINT (25.85862 63.08225),1.716407,1.1,6.970779
175,Virolahti Koivuniemi,POINT (27.67274 60.52720),2.454944,4.4,7.166146
176,Virrat Äijänneva,POINT (23.54228 62.32783),2.449347,1.1,6.588715
177,Ylitornio Meltosjärvi,POINT (24.64960 66.52952),3.015766,1.6,5.526835


In [6]:
# Project for distance calculation
weather = weather.to_crs("EPSG:3067")

In [14]:
def load_feature(file, name):
    raster = rioxarray.open_rasterio(file)
    df = (
        raster[0]       # Expect all layers to only have one channel
        .to_pandas()
        .reset_index()  # Preserve index (y, lat) during melt
        .melt(          # unpivot 2d raster -> tabular
            id_vars='y',
            var_name='long',
            value_name=name,
        ).dropna()
        .rename(columns={'y': 'lat'})
    )
    return df

all_locations = load_feature('data/wind/FIN_capacity-factor_IEC3.tif', 'power_factor')

# Find the closest station for each raster point
def closest_station(point):
    return weather.iloc[weather['geometry'].distance(point).idxmin()]
def project_wind_speed(row):
    closest = closest_station(Point(row['long'], row['lat']))
    return closest['wind_speed_pred'] / closest['wind_speed_obs'] * row['power_factor']

all_locations['power_factor_proj'] = all_locations.progress_apply(project_wind_speed, axis=1)
all_locations

100%|██████████| 12322047/12322047 [36:33<00:00, 5617.91it/s]


,lat,long,power_factor,power_factor_proj
3955,60.199519,19.085323,0.629013,0.573235
3956,60.197019,19.085323,0.629075,0.573291
3957,60.194519,19.085323,0.629133,0.573344
3958,60.192019,19.085323,0.629069,0.573286
3959,60.189519,19.085323,0.629036,0.573255
...,...,...,...,...
22474383,62.902019,31.577823,0.343220,0.312784
22478876,62.912019,31.580323,0.346518,0.315790
22478877,62.909519,31.580323,0.339815,0.309682
22478878,62.907019,31.580323,0.342411,0.312047


In [11]:
df = pd.read_csv('data/predictions/wind_projected.csv', index_col=0)
# df = all_locations.copy()
# df.to_csv('data/predictions/wind_projected.csv')

In [16]:

da = (
    df.rename(columns={"long": "x", "lat": "y"}).set_index(["y", "x"])
    .to_xarray()
)

da['power_factor_proj'].rio.to_raster("data/predictions/wind_projected.tif")
